In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL
from langchain.tools import tool

In [3]:
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()    
    return news_tool.search_by_keyword(query, k=5)

In [4]:
@tool
def python_repl_tool(
    # 매개변수 이름은 code 
    # str은 매개 변수의 타입.
    # "The python ~~ your chart." 는 매개변수에 대한 설명.
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [5]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

In [10]:
print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword
도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


In [11]:
print(f"도구 이름: {add_function.name}")
print(f"도구 설명: {add_function.description}")

도구 이름: add_function
도구 설명: Adds two numbers together.


In [12]:
tools = [search_news, python_repl_tool, add_function]

### **Agent 프롬프트 생성**

- `chat_history` : 이전 대화 내용을 저장하는 변수
- `agent_scratchpad` : 에이전트가 검색하는 과정들이나 내용 등을 임시로 저장하는 변수
- `input` : 사용자의 입력

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
# 프롬프트 생성. 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `add_function` tool for searching keyword related news."             
            "Be sure to use the 'add_function' tool when adding two numbers.",
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣기위해서 chat_history 변수
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

### **Agent 생성**

In [15]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

In [16]:
# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [17]:
# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

### **AgentExecutor**

`AgentExecutor`는 도구를 사용하는 에이전트를 실행하는 클래스입니다.
<br>

**속성**

- `agent`: 실행 루프의 각 단계에서 계획을 생성하고 행동을 결정하는 에이전트
- `tools`: 에이전트가 사용할 수 있는 유효한 도구 목록
- `return_intermediate_steps`: 최종 출력과 함께 에이전트의 중간 단계 경로를 반환할지 여부
- `max_iterations`: 실행 루프를 종료하기 전 최대 단계 수
- `max_execution_time`: 실행 루프에 소요될 수 있는 최대 시간 
- `early_stopping_method`: 에이전트가 AgentFinish를 반환하지 않을 때 사용할 조기 종료 방법. ("force" or "generate")
    - `"force"` 는 시간 또는 반복 제한에 도달하여 중지되었다는 문자열을 반환합니다.
    - `"generate"` 는 에이전트의 LLM 체인을 마지막으로 한 번 호출하여 이전 단계에 따라 최종 답변을 생성합니다.
- `handle_parsing_errors`: 에이전트의 출력 파서에서 발생한 오류 처리 방법. (True, False, 또는 오류 처리 함수)
- `trim_intermediate_steps`: 중간 단계를 트리밍하는 방법. (-1 trim 하지 않음, 또는 트리밍 함수)

**메소드**

1. `invoke`: 에이전트 실행
2. `stream`: 최종 출력에 도달하는 데 필요한 단계를 스트리밍

In [18]:
from langchain.agents import AgentExecutor

In [19]:
# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,    
    tools=tools,    
    verbose=True,   
    max_iterations=10,      
    max_execution_time=10,
    handle_parsing_errors=True,
)

In [20]:
# AgentExecutor 실행
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFAwaHQyLXo3anlpZ3RjY1BtenFYTkZXaGdqSFlpRDhDeFdoVHJXbl94cU53SklZQWQ4V05mOV82aWhBSDdnRThLbDJTNFpxSE1vQkVQM1pR?oc=5', 'content': "올해는 AI 혁신 확산의 해…新기술로 새해 투자 '청신호' - ZD넷 코리아"}, {'url': 'https://news.google.com/rss/articles/CBMiZkFVX3lxTE93UHhjaC1nWjFLUk9vNDBjd2otWUs3Q1VHNVc5OGVERENVV25hcUgtUnVwaERKRU9xaUhPaTE3aVEyb0MxaEtHOVhzSnNxQy1BaWR4ckxfVDRYWDNUUEU0eTJ5c1QtZw?oc=5', 'content': "[단독]AI가 벤처투자 심사했다…SBVA, '알파미' 국내 첫 도입 - 머니투데이"}, {'url': 'https://news.google.com/rss/articles/CBMicEFVX3lxTE1jSFNpRWx4YUZKcGUwdWlRUnZyLWJUOUJibTlENDY2UF9HcGJlMy03OXVHYnNyYWowVlpjRGQ4N2xMYkw0R2JoU1BvVXJTMnZBeWkzMFJUNWhUZHdBOVBUNkNHNHREUmxqbHN5WnJZUkjSAXRBVV95cUxOVV9xbjYtbmZiZ3gwU1VCRFpDYU1Ta0ZHMmh2S0ZSb2ZKR2VaQ2FlOGpEOEk5MHlRbTFHXzRFZ2Zrak9OOENGT1R1VE5uUkRtS3ZReWRjeElkQWNQdTc5enhtOVdJRUx0azhVbENHQnlQN0xVWQ?oc=5', 'content': "반도체 업계, AI 투자 둔화 시 '악순환' 가능성 경고 - 연합인

In [21]:
print("Agent 실행 결과:")
print(result["output"])

Agent 실행 결과:
다음은 AI 투자와 관련된 최근 뉴스입니다:

1. [올해는 AI 혁신 확산의 해…新기술로 새해 투자 '청신호'](https://news.google.com/rss/articles/CBMiVkFVX3lxTFAwaHQyLXo3anlpZ3RjY1BtenFYTkZXaGdqSFlpRDhDeFdoVHJXbl94cU53SklZQWQ4V05mOV82aWhBSDdnRThLbDJTNFpxSE1vQkVQM1pR?oc=5) - ZD넷 코리아

2. [[단독]AI가 벤처투자 심사했다…SBVA, '알파미' 국내 첫 도입](https://news.google.com/rss/articles/CBMiZkFVX3lxTE93UHhjaC1nWjFLUk9vNDBjd2otWUs3Q1VHNVc5OGVERENVV25hcUgtUnVwaERKRU9xaUhPaTE3aVEyb0MxaEtHOVhzSnNxQy1BaWR4ckxfVDRYWDNUUEU0eTJ5c1QtZw?oc=5) - 머니투데이

3. [반도체 업계, AI 투자 둔화 시 '악순환' 가능성 경고](https://news.google.com/rss/articles/CBMicEFVX3lxTE1jSFNpRWx4YUZKcGUwdWlRUnZyLWJUOUJibTlENDY2UF9HcGJlMy03OXVHYnNyYWowVlpjRGQ4N2xMYkw0R2JoU1BvVXJTMnZBeWkzMFJUNWhUZHdBOVBUNkNHNHREUmxqbHN5WnJZUkjSAXRBVV95cUxOVV9xbjYtbmZiZ3gwU1VCRFpDYU1Ta0ZHMmh2S0ZSb2ZKR2VaQ2FlOGpEOEk5MHlRbTFHXzRFZ2Zrak9OOENGT1R1VE5uUkRtS3ZReWRjeElkQWNQdTc5enhtOVdJRUx0azhVbENHQnlQN0xVWQ?oc=5) - 연합인포맥스

4. [올해 美 증시 수익률 1위는 758% ‘이 주식’…‘엔비디아 183%’ AI·코인이 달궜다 [투자360]](https://news.google.com/rss/articles/CBMi

In [22]:
# 스트리밍 모드로 실행.
result = agent_executor.stream({"input": "2 더하기 3을 알려주세요"})

# 중간 단계 출력
for step in result:    
    print(step)
    print("===" * 20)



> Entering new None chain...
{'actions': [ToolAgentAction(tool='add_function', tool_input={'a': 2, 'b': 3}, log="\nInvoking: `add_function` with `{'a': 2, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_TIZqd0xcQQJfeE6aAH1073qG', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-841c87cb-a353-4ce1-83e9-ddbe830409ab', tool_calls=[{'name': 'add_function', 'args': {'a': 2, 'b': 3}, 'id': 'call_TIZqd0xcQQJfeE6aAH1073qG', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add_function', 'args': '{"a":2,"b":3}', 'id': 'call_TIZqd0xcQQJfeE6aAH1073qG', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_TIZqd0xcQQJfeE6aAH1073qG')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_TIZqd0xcQQ

In [37]:
result = agent_executor.stream({"input": "2 더하기 3을 알려주세요"})

In [38]:
data = []

for step in result:    
    data.append(step)
    print("===" * 20)



> Entering new None chain...

Invoking: `add_function` with `{'a': 2, 'b': 3}`


5.0============================================================
2 더하기 3은 5입니다.

> Finished chain.


In [39]:
data[0]

{'actions': [ToolAgentAction(tool='add_function', tool_input={'a': 2, 'b': 3}, log="\nInvoking: `add_function` with `{'a': 2, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-98ed5694-2723-43c0-b003-cce67ddfe2da', tool_calls=[{'name': 'add_function', 'args': {'a': 2, 'b': 3}, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add_function', 'args': '{"a":2,"b":3}', 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_mdEPhZ5n2NxlWHggErpIjeQh')],
 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'function': {

In [40]:
data[1]

{'steps': [AgentStep(action=ToolAgentAction(tool='add_function', tool_input={'a': 2, 'b': 3}, log="\nInvoking: `add_function` with `{'a': 2, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-98ed5694-2723-43c0-b003-cce67ddfe2da', tool_calls=[{'name': 'add_function', 'args': {'a': 2, 'b': 3}, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add_function', 'args': '{"a":2,"b":3}', 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_mdEPhZ5n2NxlWHggErpIjeQh'), observation=5.0)],
 'messages': [FunctionMessage(content='5.0', additional_kwargs={}, response_metadata={}, name='add_function

In [41]:
data[2]

{'output': '2 더하기 3은 5입니다.',
 'messages': [AIMessage(content='2 더하기 3은 5입니다.', additional_kwargs={}, response_metadata={})]}

In [43]:
data[0]['actions'][0]

ToolAgentAction(tool='add_function', tool_input={'a': 2, 'b': 3}, log="\nInvoking: `add_function` with `{'a': 2, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-98ed5694-2723-43c0-b003-cce67ddfe2da', tool_calls=[{'name': 'add_function', 'args': {'a': 2, 'b': 3}, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add_function', 'args': '{"a":2,"b":3}', 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_mdEPhZ5n2NxlWHggErpIjeQh')

In [44]:
data[0]['messages']

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-98ed5694-2723-43c0-b003-cce67ddfe2da', tool_calls=[{'name': 'add_function', 'args': {'a': 2, 'b': 3}, 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add_function', 'args': '{"a":2,"b":3}', 'id': 'call_mdEPhZ5n2NxlWHggErpIjeQh', 'index': 0, 'type': 'tool_call_chunk'}])]